In [1]:
import numpy as np
import pandas as pd

In [2]:
# 데이터 읽어 오기 
user_cols = ['userid', 'itemid', 'timestamp']
users = pd.read_csv('../data/train/train_ratings.csv', sep=',')

directors_cols = ['itemid', 'director']
directors = pd.read_csv('../data/train/directors.tsv', sep='\t', names=directors_cols, encoding='latin-1')

genres_cols = ['itemid', 'genre']
genres = pd.read_csv('../data/train/genres.tsv', sep='\t', names=genres_cols, encoding='latin-1')

titles_cols = ['itemid', 'title']
titles = pd.read_csv('../data/train/titles.tsv', sep='\t', names=titles_cols, encoding='latin-1')

writers_cols = ['itemid', 'writer']
writers = pd.read_csv('../data/train/writers.tsv', sep='\t', names=writers_cols, encoding='latin-1')

years_cols = ['itemid', 'year']
years = pd.read_csv('../data/train/years.tsv', sep='\t', names=years_cols, encoding='latin-1')

In [3]:
users.head()

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563


In [4]:
# timestamp 제거 
users = users.drop('time', axis=1)
# rating 5로 추가
users['rating'] = 1

In [5]:
users.head()

,user,item,rating
0,11,4643,1
1,11,170,1
2,11,531,1
3,11,616,1
4,11,2140,1


In [6]:
# movie ID와 title 빼고 다른 데이터 제거
movies = titles

In [7]:
movies.head()

,itemid,title
0,item,title
1,318,"Shawshank Redemption, The (1994)"
2,2571,"Matrix, The (1999)"
3,2959,Fight Club (1999)
4,296,Pulp Fiction (1994)


In [8]:
# train, test 데이터 분리
from sklearn.model_selection import train_test_split
x = users.copy()
y = users['user']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)


In [9]:
# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))


In [10]:

# 모델별 RMSE를 계산하는 함수 
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['user'], x_test['item'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)


In [11]:

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user', columns='item', values='rating')

# train set 사용자들의 Cosine similarities 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)


In [12]:

# Neighbor size를 정해서 예측치를 계산하는 함수 
def cf_knn(user_id, movie_id, neighbor_size=0):
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기
        sim_scores = user_similarity[user_id].copy()
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기
        movie_ratings = rating_matrix[movie_id].copy()
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = movie_ratings[movie_ratings.isnull()].index
        # 현재 영화를 평가하지 않은 사용자의 rating (null) 제거
        movie_ratings = movie_ratings.drop(none_rating_idx)
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)
##### (2) Neighbor size가 지정되지 않은 경우        
        if neighbor_size == 0:          
            # 현재 영화를 평가한 모든 사용자의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
##### (3) Neighbor size가 지정된 경우
        else:                       
            # 해당 영화를 평가한 사용자가 최소 2명이 되는 경우에만 계산
            if len(sim_scores) > 1: 
                # 지정된 neighbor size 값과 해당 영화를 평가한 총사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                movie_ratings = np.array(movie_ratings)
                # 유사도를 순서대로 정렬
                user_idx = np.argsort(sim_scores)
                # 유사도를 neighbor size만큼 받기
                sim_scores = sim_scores[user_idx][-neighbor_size:]
                # 영화 rating을 neighbor size만큼 받기
                movie_ratings = movie_ratings[user_idx][-neighbor_size:]
                # 최종 예측값 계산 
                mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

In [ ]:
# 정확도 계산
score(cf_knn, neighbor_size=30)

7.470039335081935e-16

In [13]:
##### 주어진 사용자에 대해 추천을 받기 
rating_matrix = users.pivot_table(values='rating', index='user', columns='item')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)


In [14]:
def recom_movie(user_id, n_items, neighbor_size=30):
    # 현 사용자가 평가한 영화 가져오기
    user_movie = rating_matrix.loc[user_id].copy()
    for movie in rating_matrix:
        # 현 사용자가 이미 평가한 영화는 제외 (평점을 0으로)        
        if pd.notnull(user_movie.loc[movie]):
            user_movie.loc[movie] = 0
        # 현 사용자가 평가하지 않은 영화의 예상 평점 계산
        else:
            user_movie.loc[movie] = cf_knn(user_id, movie, neighbor_size)
    # 영화를 예상 평점에 따라 정렬해서 제목을 뽑아서 돌려 줌
    movie_sort = user_movie.sort_values(ascending=False)[:n_items]
    return movie_sort

In [ ]:
recom_movie(user_id=14, n_items=10, neighbor_size=30)

item
4299     5.0
59103    5.0
67429    5.0
78469    5.0
33495    5.0
3484     5.0
52462    5.0
33564    5.0
2829     5.0
71468    5.0
Name: 14, dtype: float64

In [ ]:

##### (5) 최적의 neighbor size 구하기

# train set으로 full matrix와 cosine similarity 구하기 
rating_matrix = x_train.pivot_table(values='rating', index='user', columns='item')
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)
for neighbor_size in [10, 20, 30, 40, 50, 60]:
    print("Neighbor size = %d : RMSE = %.4f" % (neighbor_size, score(cf_knn, neighbor_size)))


In [15]:
rating_df = pd.read_csv('../data/train/train_ratings.csv')
users = rating_df["user"].unique()

result = []


for user in users:
    items = recom_movie(user_id=user, n_items=10, neighbor_size=30)
    for item in items:
        result.append((user, item))

pd.DataFrame(result, columns=["user", "item"]).to_csv(
    "output/cosine_sim_submission.csv", index=False
)


KeyboardInterrupt: 